In [1]:
from models import *
from common import *

In [2]:
# estimators here
STATE = np.random.RandomState(seed=1000)
full_estimators = [rule_fit_pcc]
full_names = ['RuleFit_pcc']

In [3]:
# run 95min(2.6 GHz 6-Core Intel Core i7)
print('Full Phase Interpolation')
print("Current Prediction Order is:", data.y.columns.tolist())
print('Num of predictors:, ', data.x1.shape[1])
interpolation = Experiment(full_estimators,
                            full_names,
                            KFoldSpecial(30, shuffle=True, random_state=STATE),
                            data.x1, data.y.replace(-1.0, 0.0),
                            groups=data.comp_ids.array,
                            evaluators=[hamming_loss, error, NegLogLikelihoodEvaluator(base=2)],
                            verbose=True).run()


Full Phase Interpolation
Current Prediction Order is: ['sphere', 'worm', 'vesicle', 'other']
Num of predictors:,  23
Running experiment with 30 repetitions


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass shuffle=True, random_state=RandomState(MT19937) as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


********************

/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


**********


In [4]:
df_inter = interpolation.summary()
df_inter

,mean_train_hamming loss,std_train_hamming loss,mean_test_hamming loss,std_test_hamming loss,mean_train_error,std_train_error,mean_test_error,std_test_error,mean_train_log loss,std_train_log loss,mean_test_log loss,std_test_log loss
RuleFit_pcc,0.005766,0.003065,0.068355,0.028796,0.018639,0.009939,0.215965,0.08196,0.168415,0.0555,1.307291,0.606756


In [60]:
# run 101min(2.6 GHz 6-Core Intel Core i7)
print('Full Phase Extrapolation')
print("Current Prediction Order is:", data.y.columns.tolist())
print('Num of predictors:, ', data.x1.shape[1])
extrapolation = ExtrapolationExperiment(full_estimators, 
                            full_names,
                            GroupKFoldSpecial(len(set(data.comp_ids)), size=20),
                            data.x1, data.y.replace(-1.0, 0.0), groups=data.comp_ids.array,
                            evaluators=[hamming_loss, error, NegLogLikelihoodEvaluator(base=2),
                            GroupDescription(data.comp_descr, 'composition')],
                            verbose=True).run()

Full Phase Extrapolation
Current Prediction Order is: ['sphere', 'worm', 'vesicle', 'other']
Num of predictors:,  23
Running experiment with 28 repetitions


In [6]:
df_extra = extrapolation.summary()
df_extra

,mean_train_hamming loss,std_train_hamming loss,mean_test_hamming loss,std_test_hamming loss,mean_train_error,std_train_error,mean_test_error,std_test_error,mean_train_log loss,std_train_log loss,mean_test_log loss,std_test_log loss
RuleFit_pcc,0.005819,0.003071,0.11877,0.16098,0.019194,0.010264,0.322405,0.415679,0.168316,0.066692,2.828737,3.892979


In [7]:
import pandas as pd

df = pd.concat([df_inter, df_extra])
df.index = ['RuFit_inter_pcc', 'RuFit_extra_pcc']
df

,mean_train_hamming loss,std_train_hamming loss,mean_test_hamming loss,std_test_hamming loss,mean_train_error,std_train_error,mean_test_error,std_test_error,mean_train_log loss,std_train_log loss,mean_test_log loss,std_test_log loss
RuFit_inter_pcc,0.005766,0.003065,0.068355,0.028796,0.018639,0.009939,0.215965,0.081960,0.168415,0.055500,1.307291,0.606756
RuFit_extra_pcc,0.005819,0.003071,0.118770,0.160980,0.019194,0.010264,0.322405,0.415679,0.168316,0.066692,2.828737,3.892979
